In [1]:
import os
import kagglehub
import pandas as pd

import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import matplotlib.pyplot as plt
import subprocess
import time
import IPython.display as display
import tempfile
from sklearn.ensemble import AdaBoostClassifier
import mlflow
import mlflow.sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.model_selection import TimeSeriesSplit

## Execise 1

In this exercise, do the following:
1. Load the dataset used in the time series example - Energy consumption data. You can find it in the notebook "TSA_Example" in Time Series folder in Moodle.
2. Setup a nested MLFlow loop where different modelling experiments can be tracked and the use the dataset in point 1 to experiment and track models. You should do following combinations:
    1. At least 3 model types
    2. At least 3 different feature combinations
    3. At least 3 different options for 3 different hyperparameters
    4. At least 3 different time splits for train test
3. For each option in the combination, you should calculate & log the following in MLFlow:
    1. RMSE
    2. MAE
    3. Plot of actual vs predicted for 1 month data
    4. Plot of actual vs predicted for 1 week of data
    5. All of the combination info in point 2, such as which model, what feature combindation, what hyperparameter, what train test split has been used
4. Turn on MLFlow UI and track your experiments

1. Load the dataset used in the time series example - Energy consumption data. You can find it in the notebook "TSA_Example" in Time Series folder in Moodle.

In [2]:
# Download latest version
path = kagglehub.dataset_download("robikscube/hourly-energy-consumption")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\Jacob\.cache\kagglehub\datasets\robikscube\hourly-energy-consumption\versions\3


In [3]:
dataset_path = "C:/Users/Jacob/.cache/kagglehub/datasets/robikscube/hourly-energy-consumption/versions/3"

files = os.listdir(dataset_path)
print(files)


['AEP_hourly.csv', 'COMED_hourly.csv', 'DAYTON_hourly.csv', 'DEOK_hourly.csv', 'DOM_hourly.csv', 'DUQ_hourly.csv', 'EKPC_hourly.csv', 'est_hourly.paruqet', 'FE_hourly.csv', 'NI_hourly.csv', 'PJME_hourly.csv', 'PJMW_hourly.csv', 'pjm_hourly_est.csv', 'PJM_Load_hourly.csv']


In [4]:
file_path = "C:/Users/Jacob/.cache/kagglehub/datasets/robikscube/hourly-energy-consumption/versions/3/PJME_hourly.csv"

In [5]:
df = pd.read_csv(file_path)

In [6]:
df

,Datetime,PJME_MW
0,2002-12-31 01:00:00,26498.0
1,2002-12-31 02:00:00,25147.0
2,2002-12-31 03:00:00,24574.0
3,2002-12-31 04:00:00,24393.0
4,2002-12-31 05:00:00,24860.0
...,...,...
145361,2018-01-01 20:00:00,44284.0
145362,2018-01-01 21:00:00,43751.0
145363,2018-01-01 22:00:00,42402.0
145364,2018-01-01 23:00:00,40164.0


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145366 entries, 0 to 145365
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   Datetime  145366 non-null  object 
 1   PJME_MW   145366 non-null  float64
dtypes: float64(1), object(1)
memory usage: 2.2+ MB


In [8]:
df = df.set_index('Datetime')
df.index = pd.to_datetime(df.index)
df = df.sort_index()

In [9]:
df.head()

,PJME_MW
Datetime,
2002-01-01 01:00:00,30393.0
2002-01-01 02:00:00,29265.0
2002-01-01 03:00:00,28357.0
2002-01-01 04:00:00,27899.0
2002-01-01 05:00:00,28057.0


In [10]:
# Check for duplicate indices
duplicate_indices = df.index[df.index.duplicated(keep=False)]
duplicate_indices

DatetimeIndex(['2014-11-02 02:00:00', '2014-11-02 02:00:00',
               '2015-11-01 02:00:00', '2015-11-01 02:00:00',
               '2016-11-06 02:00:00', '2016-11-06 02:00:00',
               '2017-11-05 02:00:00', '2017-11-05 02:00:00'],
              dtype='datetime64[ns]', name='Datetime', freq=None)

In [11]:
df.loc['2014-11-02 01:00:00':'2014-11-02 03:00:00']

,PJME_MW
Datetime,
2014-11-02 01:00:00,23538.0
2014-11-02 02:00:00,23755.0
2014-11-02 02:00:00,22935.0
2014-11-02 03:00:00,22789.0


Checking for missing timestamps.

In [12]:
range_of_dates = pd.date_range(start=df.index.min(), end=df.index.max(), freq='h')
missing_timestamps = range_of_dates.difference(df.index)
missing_timestamps


DatetimeIndex(['2002-04-07 03:00:00', '2002-10-27 02:00:00',
               '2003-04-06 03:00:00', '2003-10-26 02:00:00',
               '2004-04-04 03:00:00', '2004-10-31 02:00:00',
               '2005-04-03 03:00:00', '2005-10-30 02:00:00',
               '2006-04-02 03:00:00', '2006-10-29 02:00:00',
               '2007-03-11 03:00:00', '2007-11-04 02:00:00',
               '2008-03-09 03:00:00', '2008-11-02 02:00:00',
               '2009-03-08 03:00:00', '2009-11-01 02:00:00',
               '2010-03-14 03:00:00', '2010-11-07 02:00:00',
               '2010-12-10 00:00:00', '2011-03-13 03:00:00',
               '2011-11-06 02:00:00', '2012-03-11 03:00:00',
               '2012-11-04 02:00:00', '2013-03-10 03:00:00',
               '2013-11-03 02:00:00', '2014-03-09 03:00:00',
               '2015-03-08 03:00:00', '2016-03-13 03:00:00',
               '2017-03-12 03:00:00', '2018-03-11 03:00:00'],
              dtype='datetime64[ns]', freq=None)

In [13]:
df.loc['2002-04-07 02:00:00':'2002-04-07 05:00:00']

,PJME_MW
Datetime,
2002-04-07 02:00:00,24738.0
2002-04-07 04:00:00,24487.0
2002-04-07 05:00:00,24617.0


In [14]:
# Remove all rows with duplicated indices
df = df[~df.index.duplicated(keep=False)]

Adding missing dates:

In [15]:
df=df.reindex(range_of_dates)

In [16]:
df = df.fillna(method="ffill")

C:\Users\Jacob\AppData\Local\Temp\ipykernel_22204\567689999.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method="ffill")


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 145392 entries, 2002-01-01 01:00:00 to 2018-08-03 00:00:00
Freq: h
Data columns (total 1 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   PJME_MW  145392 non-null  float64
dtypes: float64(1)
memory usage: 6.2 MB


In [18]:
df["hour"] = df.index.hour
df["day"] = df.index.day
df["month"] = df.index.month
df["weekday"] = df.index.weekday

In [19]:
df.head()

,PJME_MW,hour,day,month,weekday
2002-01-01 01:00:00,30393.0,1,1,1,1
2002-01-01 02:00:00,29265.0,2,1,1,1
2002-01-01 03:00:00,28357.0,3,1,1,1
2002-01-01 04:00:00,27899.0,4,1,1,1
2002-01-01 05:00:00,28057.0,5,1,1,1


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 145392 entries, 2002-01-01 01:00:00 to 2018-08-03 00:00:00
Freq: h
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   PJME_MW  145392 non-null  float64
 1   hour     145392 non-null  int32  
 2   day      145392 non-null  int32  
 3   month    145392 non-null  int32  
 4   weekday  145392 non-null  int32  
dtypes: float64(1), int32(4)
memory usage: 8.5 MB


In [21]:
def create_features(df):

    df = df.copy()
    # Lag features
    df['lag_1d'] = df['PJME_MW'].shift(1*24)   # 1 day lag
    df['lag_1w'] = df['PJME_MW'].shift(7*24)   # 1 week lag
    df['lag_1m'] = df['PJME_MW'].shift(30*24)  # 1 month lag

    # Rolling statistics features
    df['rolling_mean_1d'] = df['PJME_MW'].rolling(window=1*24).mean()  # Last 1 day rolling mean
    df['rolling_mean_7d'] = df['PJME_MW'].rolling(window=7*24).mean()  # Last 7 days rolling mean
    df['rolling_mean_30d'] = df['PJME_MW'].rolling(window=30*24).mean()  # Last 30 days rolling mean
    df['rolling_mean_90d'] = df['PJME_MW'].rolling(window=90*24).mean()  # Last 90 days rolling mean
    df['rolling_mean_same_month_last_year'] = df['PJME_MW'].shift(365*24).rolling(window=30*24).mean()  # Same month previous year rolling mean
    df['rolling_mean_same_week_last_year'] = df['PJME_MW'].shift(365*24).rolling(window=7*24).mean()  # Same week previous year rolling mean
    df['rolling_mean_same_day_last_year'] = df['PJME_MW'].shift(365*24).rolling(window=1*24).mean()  # Same day previous year rolling mean

    return df

df = create_features(df)

In [22]:
df.tail()

,PJME_MW,hour,day,month,weekday,lag_1d,lag_1w,lag_1m,rolling_mean_1d,rolling_mean_7d,rolling_mean_30d,rolling_mean_90d,rolling_mean_same_month_last_year,rolling_mean_same_week_last_year,rolling_mean_same_day_last_year
2018-08-02 20:00:00,44057.0,20,2,8,3,46912.0,46337.0,49244.0,39902.916667,35900.238095,36195.288889,32695.102315,36598.879167,33901.708333,37874.750000
2018-08-02 21:00:00,43256.0,21,2,8,3,45985.0,44542.0,47292.0,39789.208333,35892.583333,36189.683333,32697.885648,36594.200000,33910.625000,37660.416667
2018-08-02 22:00:00,41552.0,22,2,8,3,44094.0,42638.0,45506.0,39683.291667,35886.119048,36184.191667,32700.633333,36589.688889,33918.482143,37472.750000
2018-08-02 23:00:00,38500.0,23,2,8,3,40666.0,39276.0,42437.0,39593.041667,35881.500000,36178.723611,32703.043056,36585.406944,33926.244048,37309.500000
2018-08-03 00:00:00,35486.0,0,3,8,4,37158.0,35742.0,39340.0,39523.375000,35879.976190,36173.370833,32705.528241,36581.276389,33933.797619,37164.500000


In [23]:
# # Select the first combination
# X = df.drop(columns=["PJME_MW"])
# y = df['PJME_MW']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# # Define different feature combinations
# # Define 3 feature combinations based on your dataset
# feature_combinations = [
#     ["hour", "day", "month"],  # Combination 1: Time-based features (hour, day, month)
#     ["lag_1d", "lag_1w", "lag_1m"],  # Combination 2: Lag features (1-day, 1-week, 1-month lags)
#     ["rolling_mean_1d", "rolling_mean_7d", "rolling_mean_30d"],  # Combination 3: Rolling averages (short-term and long-term)
# ]

2. Setup a nested MLFlow loop where different modelling experiments can be tracked and the use the dataset in point 1 to experiment and track models. You should do following combinations:
    1. At least 3 model types
    2. At least 3 different feature combinations
    3. At least 3 different options for 3 different hyperparameters
    4. At least 3 different time splits for train test

In [24]:
feature_combinations = [
    ["hour", "day", "month"],  # Combination 1: Time-based features (hour, day, month)
    ["lag_1d", "lag_1w", "lag_1m"],  # Combination 2: Lag features (1-day, 1-week, 1-month lags)
    ["rolling_mean_1d", "rolling_mean_7d", "rolling_mean_30d"],  # Combination 3: Rolling averages (short-term and long-term)
]

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 145392 entries, 2002-01-01 01:00:00 to 2018-08-03 00:00:00
Freq: h
Data columns (total 15 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   PJME_MW                            145392 non-null  float64
 1   hour                               145392 non-null  int32  
 2   day                                145392 non-null  int32  
 3   month                              145392 non-null  int32  
 4   weekday                            145392 non-null  int32  
 5   lag_1d                             145368 non-null  float64
 6   lag_1w                             145224 non-null  float64
 7   lag_1m                             144672 non-null  float64
 8   rolling_mean_1d                    145369 non-null  float64
 9   rolling_mean_7d                    145225 non-null  float64
 10  rolling_mean_30d                   144673 non-null  float64
 1

In [26]:
models = {
    "RandomForest": RandomForestRegressor,
    "XGBoost": XGBRegressor,
    "LinearRegression": LinearRegression
}

In [27]:
hyperparameters = {
    "RandomForest": {
        "n_estimators": [50, 100, 200],
        "max_depth": [3, 5, 7]
    },
    "XGBoost": {
        "n_estimators": [50, 100, 200],
        "max_depth": [3, 5, 7]
    },
    "LinearRegression": {'copy_X': [True,False], 
               'fit_intercept': [True,False], 
               'n_jobs': [1,5,10,15,None], 
               'positive': [True,False]
               }
}

In [ ]:

time_splits = [
    (0.8, 0.2),  # 80% train, 20% test
    (0.7, 0.3),
    (0.9, 0.1)
]

In [29]:
mlflow.set_experiment('model_tracking_experiment')
results = []


In [ ]:
for model_name, model_class in models.items():
    for features in feature_combinations:
        X = df[features]
        y = df['PJME_MW']
        

        if model_name == 'RandomForest':
            n_estimators_values = hyperparameters['RandomForest']['n_estimators']
            max_depth_values = hyperparameters['RandomForest']['max_depth']
            
            for n_estimators in n_estimators_values:
                for max_depth in max_depth_values:
                    hyperparameter_set = {'n_estimators': n_estimators, 'max_depth': max_depth, 'n_jobs': -1}
                    
        elif model_name == 'XGBoost':
            n_estimators_values = hyperparameters['XGBoost']['n_estimators']
            max_depth_values = hyperparameters['XGBoost']['max_depth']
            
            for n_estimators in n_estimators_values:
                for max_depth in max_depth_values:
                    hyperparameter_set = {'n_estimators': n_estimators, 'max_depth': max_depth, 'n_jobs': -1}
                    
        elif model_name == 'LinearRegression':
            fit_intercept_values = hyperparameters['LinearRegression']['fit_intercept']
            
            for fit_intercept in fit_intercept_values:
                hyperparameter_set = {'fit_intercept': fit_intercept}
                    

        for train_size, test_size in time_splits:

            train, test = train_test_split(df, train_size=train_size, shuffle=False)
            X_train, y_train = train[features], train['PJME_MW']
            X_test, y_test = test[features], test['PJME_MW']


            if model_name == 'LinearRegression':
                X_train = X_train.dropna()
                y_train = y_train.loc[X_train.index]
                X_test = X_test.dropna()
                y_test = y_test.loc[X_test.index]



            with mlflow.start_run():

                model = model_class(**hyperparameter_set)
                model.fit(X_train, y_train)
                
                
                y_pred = model.predict(X_test)
                
    
                rmse = np.sqrt(mean_squared_error(y_test, y_pred))
                mae = mean_absolute_error(y_test, y_pred)
                

                plt.figure(figsize=(10, 6))
                plt.plot(y_test[:30], label='Actual') 
                plt.plot(y_pred[:30], label='Predicted')
                plt.title('Actual vs Predicted (1 Month)')
                plt.legend()
                plt.savefig('actual_vs_predicted_1_month.png')
                plt.close()
                

                plt.figure(figsize=(10, 6))
                plt.plot(y_test[:7], label='Actual')
                plt.plot(y_pred[:7], label='Predicted')
                plt.title('Actual vs Predicted (1 Week)')
                plt.legend()
                plt.savefig('actual_vs_predicted_1_week.png')
                plt.close()

                mlflow.log_param('Model', model_name)
                mlflow.log_param('Features', features)
                mlflow.log_param('Hyperparameters', hyperparameter_set)
                mlflow.log_param('Train/Test Split', f'{train_size}/{test_size}')
                mlflow.log_metric('RMSE', rmse)
                mlflow.log_metric('MAE', mae)
                mlflow.log_artifact('actual_vs_predicted_1_month.png')
                mlflow.log_artifact('actual_vs_predicted_1_week.png')


                mlflow.sklearn.log_model(model, model_name)

                results.append({
                    "features": features,
                    "model": model_name,
                    "hyperparameters": hyperparameter_set,
                    "Train/test split": f'{train_size}/{test_size}',
                    "Rmse": rmse,
                    "MAE":mae,
            })

results_df = pd.DataFrame(results)
print("\nSummary of Experiment Results:")
print(results_df)


2025/02/27 16:45:13 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/02/27 16:45:19 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/02/27 16:45:25 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/02/27 16:45:34 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/02/27 16:45:44 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/02/27 16:45:56 


Summary of Experiment Results:
                                             features             model  \
0                                  [hour, day, month]      RandomForest   
1                                  [hour, day, month]      RandomForest   
2                                  [hour, day, month]      RandomForest   
3                            [lag_1d, lag_1w, lag_1m]      RandomForest   
4                            [lag_1d, lag_1w, lag_1m]      RandomForest   
5                            [lag_1d, lag_1w, lag_1m]      RandomForest   
6   [rolling_mean_1d, rolling_mean_7d, rolling_mea...      RandomForest   
7   [rolling_mean_1d, rolling_mean_7d, rolling_mea...      RandomForest   
8   [rolling_mean_1d, rolling_mean_7d, rolling_mea...      RandomForest   
9                                  [hour, day, month]           XGBoost   
10                                 [hour, day, month]           XGBoost   
11                                 [hour, day, month]           XGBo

In [34]:
# Start MLFlow UI from the notebook
print("\nStarting MLFlow UI...")
process = subprocess.Popen(["mlflow", "ui", "--port", "5000"])  # Starts MLFlow UI on port 5000

# Wait a moment to ensure the server starts
time.sleep(3)

# Display a link to the MLFlow UI
display.display(display.Markdown("[Open MLFlow UI](http://localhost:5000)"))

print("MLFlow UI is running at http://localhost:5000. Press Ctrl+C in the terminal to stop it.")


Starting MLFlow UI...


[Open MLFlow UI](http://localhost:5000)

MLFlow UI is running at http://localhost:5000. Press Ctrl+C in the terminal to stop it.


In [32]:
# # Start MLFlow experiment
# mlflow.set_experiment("Energy_consumption_D&T")

# results = []

# # Train models with different feature combinations
# for features in feature_combinations:
#     for model_name, modelClass in models.items():  # Ensure model_name is defined before use
#         # Remove lag and roll features only for LinearRegression
#         X_selected = df[features] if model_name != "LinearRegression" else df[[f for f in features if "lag" not in f and "roll" not in f]]
#         X_selected = X_selected.dropna()  # Remove NaN rows
#         y = df.loc[X_selected.index, "PJME_MW"]  # Ensure y aligns with filtered X

#         model_hyperparameters = hyperparameters[model_name]  # Get hyperparameters

#         for param_set in zip(*model_hyperparameters.values()): 
#             param_dict = dict(zip(model_hyperparameters.keys(), param_set))
            
#             for train_idx, test_idx in tscv.split(X_selected):  # Fix split to use correct X
#                 X_train, X_test = X_selected.iloc[train_idx], X_selected.iloc[test_idx]
#                 y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

#                 # Initialize the model with appropriate hyperparameters
#                 if model_name == "LinearRegression":
#                     model = modelClass(copy_X=param_dict["copy_X"],
#                                        fit_intercept=param_dict["fit_intercept"], 
#                                        n_jobs=param_dict["n_jobs"], 
#                                        positive=param_dict["positive"])
#                 else:
#                     model = modelClass(n_estimators=param_dict["n_estimators"], 
#                                        max_depth=param_dict["max_depth"])

#                 with mlflow.start_run():
#                     mlflow.log_param("features", features)
#                     mlflow.log_param("model_type", model_name)
#                     mlflow.log_param("hyperparameters", param_dict)
                    
#                     # Train the model
#                     model.fit(X_train, y_train)
#                     predictions = model.predict(X_test)
                    
#                     # Calculate metrics
#                     rmse = np.sqrt(mean_squared_error(y_test, predictions))
#                     mae = mean_absolute_error(y_test, predictions)

#                     # Log metrics
#                     mlflow.log_metric("RMSE", rmse)
#                     mlflow.log_metric("MAE", mae)

#                     # Store results for summary
#                     results.append({
#                         "model": model_name,
#                         "feature_set": features,
#                         "hyperparameters": param_dict,
#                         "RMSE": rmse,
#                         "MAE": mae
#                     })

# # Print summary
# results_df = pd.DataFrame(results)
# print("\nSummary of Experiment Results:")
# print(results_df)


In [33]:

# hyperparameters = {
#     "RandomForest": [
#         {"n_estimators": 50, "max_depth": 10, "min_samples_split": 2},
#         {"n_estimators": 100, "max_depth": 20, "min_samples_split": 4},
#         {"n_estimators": 150, "max_depth": None, "min_samples_split": 8},
#     ],
#     "LogisticRegression": [
#         {"max_iter": 100, "C": 0.1, "solver": "lbfgs"},
#         {"max_iter": 200, "C": 1.0, "solver": "liblinear"},
#         {"max_iter": 300, "C": 10, "solver": "saga"},
#     ]
# }



# # Start MLFlow experiment
# mlflow.set_experiment("MLFlow Jacob")

# results = []

# # Train models with different feature combinations
# for features in feature_combinations:
#     for model_name, model in zip(["RandomForest", "LogisticRegression", "KNN"], 
#                                  [RandomForestClassifier(random_state=42), LogisticRegression(max_iter=200)]):
#         with mlflow.start_run():
#             # Log feature combination and model type
#             mlflow.log_param("features", features)
#             mlflow.log_param("model_type", model_name)
            
#             # Train the model
#             model.fit(X_train[features], y_train)
#             predictions = model.predict(X_test[features])
            
#             # Calculate metrics
#             accuracy = accuracy_score(y_test, predictions)
#             precision = precision_score(y_test, predictions, average='weighted')
#             recall = recall_score(y_test, predictions, average='weighted')
#             f1 = f1_score(y_test, predictions, average='weighted')
            
#             # Log metrics
#             mlflow.log_metric("accuracy", accuracy)
#             mlflow.log_metric("precision", precision)
#             mlflow.log_metric("recall", recall)
#             mlflow.log_metric("f1_score", f1)
            
#             # Create and log a plot of the metrics
#             fig, ax = plt.subplots(figsize=(8, 4))
#             metrics = [accuracy, precision, recall, f1]
#             metric_names = ["Accuracy", "Precision", "Recall", "F1 Score"]
#             ax.bar(metric_names, metrics, color='skyblue')
#             ax.set_title(f"{model_name} Metrics for Feature Set: {features}")
#             ax.set_ylim(0, 1)
            
#             # Save the plot to a temporary file and log it as an artifact
#             temp_file = tempfile.NamedTemporaryFile(suffix=".png", delete=False)
#             plt.savefig(temp_file.name)
#             mlflow.log_artifact(temp_file.name, artifact_path="plots")
#             temp_file.close()
            
#             # Log the model
#             mlflow.sklearn.log_model(model, model_name)
            
#             # Store results for summary
#             results.append({
#                 "features": features,
#                 "model": model_name,
#                 "accuracy": accuracy,
#                 "precision": precision,
#                 "recall": recall,
#                 "f1_score": f1
#             })

# # Print summary of results
# results_df = pd.DataFrame(results)
# print("\nSummary of Experiment Results:")
# print(results_df)